In [1]:
import pandas as pd
from alfabet_lite import model

2024-10-16 18:38:18.575776: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-10-16 18:38:18.575805: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-16 18:38:18.575810: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-16 18:38:18.575867: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-16 18:38:18.575901: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Load the test dataset

In [2]:
test_data = pd.read_csv('test_data.csv.gz')
test_data.head()

,molecule,bond_index,fragment1,fragment2,bde,bond_type,delta_stereo
0,C#C/C(C)=C/CNCC,1,[C]#C,C[C]=CCNCC,132.918956,C-C,0.0
1,C#C/C(C)=C/CNCC,2,C#C/[C]=C/CNCC,[CH3],90.545156,C-C,0.0
2,C#C/C(C)=C/CNCC,4,[CH2]NCC,[CH]=C(C)C#C,92.585188,C-C,0.0
3,C#C/C(C)=C/CNCC,5,C#C/C(C)=C/[CH2],CC[NH],62.904012,C-N,0.0
4,C#C/C(C)=C/CNCC,6,C#C/C(C)=C/C[NH],[CH2]C,83.992707,C-N,0.0


In this test dataset and in the alfabet predictions, `bond_index` corresponds to the bond index assinged by RDKit for a molecule with explicit hydrogens

In [3]:
from rdkit import Chem
molH = Chem.AddHs(Chem.MolFromSmiles('C#C/C(C)=C/CNCC'))
bond = molH.GetBondWithIdx(5)
print(f'{bond.GetBeginAtom().GetSymbol()}-{bond.GetEndAtom().GetSymbol()}')

C-N


Get a list of unique molecules in the test dataset

In [4]:
test_molecules = test_data.molecule.unique()
test_molecules[:5]

array(['C#C/C(C)=C/CNCC', 'C#C/C=C/C(=O)O', 'C#C/C=C/C=C/C=C/C',
       'C#C/C=C/COCCCC', 'C#CC#CCCO'], dtype=object)

`model.predict(...)` expects a list-like object of SMILES strings. The longest part of the calculation is breaking the bonds in the molecule and generating radical SMILES, which uses the `joblib` library to parallelize these calculations across multiple cores

In [5]:
alfabet_predictions = model.predict(test_molecules, verbose=False)

2024-10-16 18:38:31.678312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-16 18:39:37.255292: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8536092556285174198
2024-10-16 18:39:37.255311: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 673847468907914104
2024-10-16 18:39:37.255316: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10036925860258481506
2024-10-16 18:39:37.255321: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2088792219918125148
2024-10-16 18:39:37.255327: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 34662923092682800
2024-10-16 18:39:37.255331: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

In [6]:
alfabet_predictions.head()

,molecule,bond_index,bond_type,fragment1,fragment2,is_valid_stereo,bde_pred,bdfe_pred,is_valid,bde,bdfe,set
0,C#C/C(C)=C/CNCC,1,C-C,[C]#C,C[C]=CCNCC,False,134.833633,125.144165,True,NaN,NaN,NaN
1,C#C/C(C)=C/CNCC,2,C-C,C#C/[C]=C/CNCC,[CH3],True,91.219055,76.888222,True,90.545156,76.548568,train
2,C#C/C(C)=C/CNCC,4,C-C,[CH2]NCC,[CH]=C(C)C#C,True,92.738388,79.602707,True,92.585188,79.312118,train
3,C#C/C(C)=C/CNCC,5,C-N,C#C/C(C)=C/[CH2],CC[NH],True,62.973671,49.733555,True,62.904012,50.027528,train
4,C#C/C(C)=C/CNCC,6,C-N,C#C/C(C)=C/C[NH],[CH2]C,True,83.867477,69.303635,True,83.992707,69.312762,train
